In [15]:
import tweepy as tw
import pandas as pd
import numpy as np
import json
import re

In [7]:
consumer_key= 'insert'
consumer_secret= 'insert'
access_token= 'insert'
access_token_secret= 'insert'

auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth,wait_on_rate_limit=True,wait_on_rate_limit_notify=True)

In [8]:
public_tweets = api.search('marks and sparks -filter:retweets', count=2000)

In [9]:
my_list_of_dicts = []
for each_json_tweet in public_tweets:
    my_list_of_dicts.append(each_json_tweet._json)

    
with open('tweet_json_Data.txt', 'w') as file:
        file.write(json.dumps(my_list_of_dicts, indent=4))

        
my_demo_list = []
with open('tweet_json_Data.txt', encoding='utf-8') as json_file:  
    all_data = json.load(json_file)
    for each_dictionary in all_data:
        tweet_id = each_dictionary['id']
        text = each_dictionary['text']
        favorite_count = each_dictionary['favorite_count']
        retweet_count = each_dictionary['retweet_count']
        created_at = each_dictionary['created_at']
        my_demo_list.append({'tweet_id': str(tweet_id),
                             'text': str(text),
                             'favorite_count': int(favorite_count),
                             'retweet_count': int(retweet_count),
                             'created_at': created_at,
                            })
        
        tweet_dataset = pd.DataFrame(my_demo_list, columns = 
                                  ['tweet_id', 'text', 
                                   'favorite_count', 'retweet_count', 
                                   'created_at'])


 #Writing tweet dataset ti csv file for future reference
tweet_dataset.to_csv('sparks.csv')

In [23]:
from google.colab import files
files.download("sparks.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [20]:
def cleanUpTweet(txt):
  txt = re.sub(r'@[A-Za-z0-9_]+', '',txt)
  txt = re.sub(r'#', '', txt)
  txt = re.sub(r'RT : ', '', txt)
  txt = re.sub(r'https?:\/\/[A-Aa-a0-9\.\/]+', '', txt)
  txt = re.sub(r"http\S+", "", txt) #removed URL!

  return txt

tweet_dataset["clean"] = tweet_dataset["text"].map(lambda x: cleanUpTweet(x))


In [24]:
tweet_dataset.head()

,tweet_id,text,favorite_count,retweet_count,created_at,clean
0,1572652592945319936,I’ve been finding this too with salad stuff(ev...,1,0,Wed Sep 21 18:22:51 +0000 2022,I’ve been finding this too with salad stuff(ev...
1,1572529441162821632,"I realized I couldn't let them go, even though...",0,0,Wed Sep 21 10:13:29 +0000 2022,"I realized I couldn't let them go, even though..."
2,1572522169728667648,Just peaked the sales assistant in Marks and S...,10,2,Wed Sep 21 09:44:35 +0000 2022,Just peaked the sales assistant in Marks and S...
3,1572498829232177154,Wahhhhhh!\n🎄🎄🎄🎄🎄🎄🎄🎄🎄🎄\nMarks n Sparks have Chr...,5,0,Wed Sep 21 08:11:51 +0000 2022,Wahhhhhh!\n🎄🎄🎄🎄🎄🎄🎄🎄🎄🎄\nMarks n Sparks have Chr...
4,1572299298506776578,Marks and Spencer sparks mass online debate ov...,0,0,Tue Sep 20 18:58:59 +0000 2022,Marks and Spencer sparks mass online debate ov...


In [ ]:
tweet_dataset['data'] = tweet_dataset.clean.tolist()

In [ ]:
tweet_dataset = tweet_dataset.clean.tolist()

tweet_dataset['data'] = [re.sub('\s*@\s*\s?', '', str(sent)) for sent in tweet_dataset['data']]
tweet_dataset['data'] = [re.sub('\?', '', str(sent)) for sent in tweet_dataset['data']]
tweet_dataset['data'] = [re.sub('\_', '', str(sent)) for sent in tweet_dataset['data']]
tweet_dataset['data'] = [re.sub('@"[\d-]"', '', str(sent)) for sent in tweet_dataset['data']]

#remove new lines characters?
tweet_dataset['data'] = [re.sub('\s+', '', str(sent)) for sent in tweet_dataset['data']]

tweet_dataset['data'] = tweet_dataset['data'].str.lower

In [ ]:
!pip install pyLDAvis

In [34]:
from __future__ import print_function

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.models import CoherenceModel

import spacy

from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk.stem.lancaster import LancasterStemmer

import operator

np.random.seed(2018)
import sys
import nltk
import csv, os

import pyLDAvis
import pyLDAvis.gensim_models

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [35]:
def sent_to_words(sentences):
  for sentence in sentences :
    yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

data_words = list(sent_to_words(tweet_dataset))

print(data_words[:3])

[['ve', 'been', 'finding', 'this', 'too', 'with', 'salad', 'stuff', 'even', 'from', 'marks', 'and', 'sparks', 'not', 'even', 'lasting', 'dy'], ['realized', 'couldn', 'let', 'them', 'go', 'even', 'though', 'some', 'of', 'them', 'had', 'album', 'marks', 'and', 'minor', 'scratches', 'the', 'sparks', 'are', 'still', 'strong'], ['just', 'peaked', 'the', 'sales', 'assistant', 'in', 'marks', 'and', 'sparks', 'good', 'morning', 'of', 'terfing']]


In [36]:
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)


lemmatized_texts = lemmatization(tweet_dataset)
print (lemmatized_texts[:2])

['’ve find too salad even last dy 🤯', 'realize let go even album mark minor scratch still strong']


In [37]:
bigram_phrases = gensim.models.Phrases(data_words, min_count=5, threshold=100)
trigram_phrases = gensim.models.Phrases(bigram_phrases[data_words], threshold=100)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
  return [bigram[doc] for doc in texts]

def make_trigrams(texts):
  return [trigram[bigram[doc]] for doc in texts]

data_bigrams = make_bigrams(data_words)
data_bigrams_trigrams = make_trigrams(data_bigrams)

print(data_bigrams_trigrams[2])

['just', 'peaked', 'the', 'sales', 'assistant', 'in', 'marks', 'and', 'sparks', 'good', 'morning', 'of', 'terfing']


/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [38]:
from gensim.models import TfidfModel

id2word = corpora.Dictionary(data_bigrams_trigrams)

texts = data_bigrams_trigrams

corpus = [id2word.doc2bow(text) for text in texts]
# print (corpus[0][0:20])

tfidf = TfidfModel(corpus, id2word=id2word)

low_value = 0.03
words  = []
words_missing_in_tfidf = []
for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = [] #reinitialize to be safe. You can skip this.
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops = low_value_words+words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # The words with tf-idf socre 0 will be missing

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    corpus[i] = new_bow

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")

In [43]:
lda_model.print_topics()

[(0,
  '0.025*"another" + 0.025*"woolies" + 0.025*"even" + 0.013*"the" + 0.013*"it" + 0.013*"song" + 0.013*"rest" + 0.013*"of" + 0.013*"be" + 0.013*"but"'),
 (1,
  '0.043*"and" + 0.023*"the" + 0.023*"in" + 0.023*"of" + 0.023*"just" + 0.023*"morning" + 0.023*"peaked" + 0.023*"sales" + 0.023*"terfing" + 0.023*"assistant"'),
 (2,
  '0.021*"holly" + 0.021*"for" + 0.021*"haruto" + 0.021*"with" + 0.011*"lovely" + 0.011*"have" + 0.011*"po" + 0.011*"here" + 0.011*"anyone" + 0.011*"guys"'),
 (3,
  '0.028*"it" + 0.028*"the" + 0.028*"this" + 0.028*"of" + 0.028*"cheese" + 0.015*"from" + 0.015*"is" + 0.015*"to" + 0.015*"two" + 0.015*"come"'),
 (4,
  '0.030*"for" + 0.030*"and" + 0.023*"or" + 0.015*"to" + 0.015*"from" + 0.015*"of" + 0.015*"you" + 0.015*"that" + 0.015*"amp" + 0.015*"like"'),
 (5,
  '0.046*"and" + 0.028*"the" + 0.019*"to" + 0.019*"on" + 0.019*"belt" + 0.019*"had" + 0.019*"it" + 0.019*"still" + 0.019*"them" + 0.010*"with"'),
 (6,
  '0.061*"and" + 0.037*"you" + 0.025*"like" + 0.025*"your

In [44]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds="mmds", R=25)
vis

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.004600  0.166559       1        1  19.210261
5      0.005177 -0.184312       2        1  14.141668
9     -0.173862 -0.087651       3        1  13.997830
2     -0.155202  0.141447       4        1  12.514703
0      0.128351 -0.101211       5        1   9.663372
6      0.161749  0.006152       6        1   9.375869
3      0.120882  0.115720       7        1   8.004417
7     -0.038682 -0.075522       8        1   5.469323
8     -0.067981  0.025429       9        1   4.545097
1      0.014968 -0.006611      10        1   3.077460, topic_info=       Term       Freq      Total Category  logprob  loglift
0       and  15.000000  15.000000  Default  25.0000  25.0000
29      the  10.000000  10.000000  Default  24.0000  24.0000
34       in   6.000000   6.000000  Default  23.0000  23.0000
46       my   2.000000   2.000000  Default  22.0000  22.0000
44     have   2.000000   2.000000  Default  21.0000  21.0000
..      ...        ...        ...      ...      ...      ...
269      xd   0.034823   1.340038  Topic10  -6.1698  -0.1691
242  entire   0.034812   1.340051  Topic10  -6.1701  -0.1694
244     our   0.034788   1.340081  Topic10  -6.1709  -0.1702
52      for   0.036159   6.258026  Topic10  -6.1322  -1.6726
105     now   0.035319   3.874881  Topic10  -6.1557  -1.2168

[386 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
246       7  0.863874     after
16        2  0.779245     album
130       1  0.291666       all
130       3  0.583333       all
107       3  0.779831    always
...     ...       ...       ...
85        5  0.151712       you
85        6  0.303424       you
86        4  0.405035      your
86        6  0.405035      your
194       2  0.779244  yourself

[345 rows x 3 columns], R=25, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 6, 10, 3, 1, 7, 4, 8, 9, 2])